In [2]:
# 🔧 Установка необходимых библиотек
!pip install snntorch tslearn --quiet

# 📚 Импорты
import torch
import torch.nn as nn
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch import utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np

# 📊 Загрузка ECG5000 (анализ временных рядов)
from tslearn.datasets import UCR_UEA_datasets
ucr = UCR_UEA_datasets()
X, y, _, _ = ucr.load_dataset("ECG5000")

# 📊 Загрузка ECG5000 (анализ временных рядов)
from tslearn.datasets import UCR_UEA_datasets
ucr = UCR_UEA_datasets()
X, y, _, _ = ucr.load_dataset("ECG5000")

# Преобразуем к нужному формату
X = X.squeeze()
y = y.astype(int)

# ⚙️ Бинарная классификация: 1 = норма, остальные = аномалия
y_binary = (y != 1).astype(int)

# ⚖️ Нормализация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 🔀 Разделение
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# 🧠 Преобразование в тензоры
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Добавим размерность времени: просто дублируем вход
time_steps = 25
X_train_seq = X_train.unsqueeze(1).repeat(1, time_steps, 1)
X_test_seq = X_test.unsqueeze(1).repeat(1, time_steps, 1)

# 📦 DataLoader
train_data = torch.utils.data.TensorDataset(X_train_seq, y_train)
test_data = torch.utils.data.TensorDataset(X_test_seq, y_test)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

# 🔧 SNN модель
class SNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.lif1 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())
        self.fc2 = nn.Linear(64, 1)
        self.lif2 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        spk2_rec = []

        for step in range(x.size(1)):
            cur1 = self.fc1(x[:, step])
            spk1, mem1 = self.lif1(cur1, mem1)

            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)

        return torch.stack(spk2_rec, dim=1).sum(1)

# ⚙️ Инициализация модели
model = SNN()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 🎯 Обучение
for epoch in range(10):
    total_loss = 0
    for x, y in train_loader:
        out = model(x).squeeze()
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# ✅ Тестирование
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for x, y in test_loader:
        out = model(x).squeeze()
        preds = (torch.sigmoid(out) > 0.5).int()
        all_preds.extend(preds.tolist())
        all_labels.extend(y.tolist())

# 📊 Метрики
print(classification_report(all_labels, all_preds, digits=4))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 31.3 MB/s eta 0:00:00
Epoch 1, Loss: 3.7646
Epoch 2, Loss: 2.7482
Epoch 3, Loss: 2.9723
Epoch 4, Loss: 2.8704
Epoch 5, Loss: 2.9678
Epoch 6, Loss: 2.9028
Epoch 7, Loss: 2.9352
Epoch 8, Loss: 2.8052
Epoch 9, Loss: 2.9677
Epoch 10, Loss: 2.8379
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000        58
         1.0     1.0000    1.0000    1.0000        42

    accuracy                         1.0000       100
   macro avg     1.0000    1.0000    1.0000       100
weighted avg     1.0000    1.0000    1.0000       100

